# EasyCV度量学习

本文将介绍如何使用EasyCV快速使用Resnet50进行度量学习模型的训练、推理

## 安装依赖包

注: 在PAI-DSW docker中无需安装相关依赖，可跳过此部分 在本地notebook环境中执行


1. 首先，安装pytorch和对应版本的torchvision，支持Pytorch1.5.1以上版本

In [ ]:
# install pytorch and torch vision
! conda install --yes pytorch==1.10.0 torchvision==0.11.0 -c pytorch

2. 获取torch和cuda版本，安装对应版本的mmcv和nvidia-dali

In [ ]:
import torch
import os
os.environ['CUDA']='cu' + torch.version.cuda.replace('.', '')
os.environ['Torch']='torch'+torch.version.__version__.replace('+PAI', '')
!echo $CUDA
! echo $Torch

In [ ]:
# install some python deps
! pip install mmcv-full==1.4.4 -f https://download.openmmlab.com/mmcv/dist/${CUDA}/${Torch}/index.html
! pip install http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/third_party/nvidia_dali_cuda100-0.25.0-1535750-py3-none-manylinux2014_x86_64.whl

3.  安装EasyCV算法包

In [ ]:
pip install pai-easycv

## 简单验证

In [ ]:
from easycv.apis import *

## 运行环境要求

PAI-Pytorch 1.7/1.8镜像， GPU机型 P100 or V100， 内存 32G

## CUB200 度量学习

下面示例介绍如何利用[CUB200](http://www.vision.caltech.edu/datasets/cub_200_2011/)数据，使用ResNet50模型快速进行度量学习模型的训练评估、模型预测过程

### 数据准备
下载cub200数据，解压到`data/cub200`目录， 目录结构如下

```text
data/cub200
├── images
├── images.txt
├── image_class_labels.txt
├── train_test_split.txt
```

In [ ]:
! mkdir -p data/cub200 && wget https://data.caltech.edu/tindfiles/serve/1239ea37-e132-42ee-8c09-c383bb54e7ff/ &&  tar -zxf CUB_200_2011.tgz -C data/cub200/

### 训练模型
下载训练配置文件

In [ ]:
! rm -rf cub_resnet50_jpg.py
!wget https://raw.githubusercontent.com/alibaba/EasyCV/master/configs/metric_learning/cub_resnet50_jpg.py

使用单卡gpu进行训练和验证集评估

In [ ]:
! python -m easycv.tools.train  cub_resnet50_jpg.py --work_dir work_dirs/metric_learning/cub/r50

### 导出模型

模型训练完成，使用export命令导出模型进行推理，导出的模型包含推理时所需的预处理信息、后处理信息

In [ ]:
# 查看训练产生的pt文件
! ls  work_dirs/metric_learning/cub/r50*

ClsEvaluator_neck_top1_best.pth 是训练过程中产生的acc最高的pth,导出该模型

In [ ]:
! python -m easycv.tools.export cub_resnet50_jpg.py work_dirs/metric_learning/cub/r50/ClsEvaluator_neck_top1_best.pth  work_dirs/metric_learning/cub/r50/best_export.pth

### 预测
下载测试图片

In [ ]:
! wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/cifar10/qince_data/predict/aeroplane_s_000004.png

In [ ]:
import cv2
from easycv.predictors.classifier import TorchClassifier

output_ckpt = 'work_dirs/metric_learning/cub/epoch_90_export.pth'
tcls = TorchClassifier(output_ckpt, topk=1)

img = cv2.imread('aeroplane_s_000004.png')
# input image should be RGB order
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
output = tcls.predict([img])
print(output)